In [1]:
from langchain_google_genai import GoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

c:\Users\Administrator\crm-venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Connect to Gemini llm

In [2]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
llm("What is an AI assistant")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14428\3708976949.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  llm("What is an AI assistant")


'An AI assistant is a type of artificial intelligence (AI) software that can perform tasks or provide information to users. AI assistants are typically powered by machine learning and natural language processing (NLP) technologies, and they can be used for a wide range of tasks, including:\n\n* **Answering questions:** AI assistants can answer questions about a variety of topics, from the weather to the latest news.\n* **Providing information:** AI assistants can provide information on a variety of topics, such as the location of a restaurant or the definition of a word.\n* **Performing tasks:** AI assistants can perform tasks such as setting alarms, sending emails, and playing music.\n* **Helping with decision-making:** AI assistants can help users make decisions by providing information and recommendations.\n\nAI assistants are becoming increasingly popular as they become more sophisticated and capable. They are now used by millions of people around the world for a variety of tasks.\

## Connect with database and ask some basic question

In [4]:

from langchain.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_experimental.sql import SQLDatabaseChain

In [5]:
db_name=os.getenv("PG_DB")
user=os.getenv("PG_USERNAME")
password=os.getenv("PG_PASSWORD")
host=os.getenv("PG_HOST")
port=os.getenv("PG_PORT")

In [6]:
db = SQLDatabase.from_uri(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}")

In [7]:
print(db.table_info)


CREATE TABLE "AdTemplates" (
	id SERIAL NOT NULL, 
	"merchantId" INTEGER, 
	name VARCHAR(255), 
	leads INTEGER, 
	"createdAt" TIMESTAMP WITH TIME ZONE NOT NULL, 
	"updatedAt" TIMESTAMP WITH TIME ZONE NOT NULL, 
	"productId" INTEGER, 
	CONSTRAINT "AdTemplates_pkey" PRIMARY KEY (id), 
	CONSTRAINT "AdTemplates_merchantId_fkey" FOREIGN KEY("merchantId") REFERENCES "Merchants" (id) ON DELETE CASCADE ON UPDATE CASCADE, 
	CONSTRAINT fk_product_id FOREIGN KEY("productId") REFERENCES "Products" (id)
)

/*
3 rows from AdTemplates table:
id	merchantId	name	leads	createdAt	updatedAt	productId
52	2	buy_action	None	2024-01-14 07:36:30.007000-08:00	2024-01-14 07:36:31.165000-08:00	9
46	2	buy_action	None	2024-01-08 05:55:48.115000-08:00	2024-01-08 05:55:49.352000-08:00	1
47	2	buy_action	None	2024-01-08 10:30:35.620000-08:00	2024-01-08 10:30:36.736000-08:00	3
*/


CREATE TABLE "Ads" (
	id SERIAL NOT NULL, 
	"merchantId" INTEGER, 
	"templateId" INTEGER, 
	read INTEGER, 
	delivered INTEGER, 
	sent INTEG

In [14]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)

# create a query
response = chain.invoke({"question":   'How many customers are there? Remember to add public."<table name>" for postgreql. inluding the quotes on table name. And return only Just the postgreSQL query'})
response

'```sql\nSELECT\n  COUNT(*)\nFROM public."Customers";\n```'

Alternative to create an sql agent

In [15]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [16]:
from langchain.agents import create_sql_agent 
from langchain.agents import AgentExecutor 
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [17]:
cleaned_query = response.strip('```sql\n').strip('\n```')
cleaned_query

'SELECT\n  COUNT(*)\nFROM public."Customers";'

In [18]:
db.run(cleaned_query)

'[(4386,)]'

In [25]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)

def execute_query(question):
    try:
        # Geerate SQL query from question
        response = chain.invoke({"question":   question + ' Remember to add public."<table name>" for postgreql including the quotes on table name'})
        print(response)
        print("########################")
        # clean the query
        cleaned_query = response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        print("########################")
        result = db.run(cleaned_query)
        print("########################")
        print(result)
    except Exception as e:
        print(f"An error occured", {e})
        return False

In [26]:
question = "What is the number of products sold per category?"

execute_query(question)

```sql
SELECT
  COUNT(category.name),
  category.name
FROM public."OrderItems"
JOIN public."Products"
  ON "OrderItems"."productId" = public."Products".id
JOIN public."Categories"
  ON public."Products"."categoryId" = public."Categories".id
GROUP BY
  category.name;
```
########################
SELECT
  COUNT(category.name),
  category.name
FROM public."OrderItems"
JOIN public."Products"
  ON "OrderItems"."productId" = public."Products".id
JOIN public."Categories"
  ON public."Products"."categoryId" = public."Categories".id
GROUP BY
  category.name;
########################
An error occured {ProgrammingError('(psycopg2.errors.UndefinedTable) missing FROM-clause entry for table "category"\nLINE 2:   COUNT(category.name),\n                ^\n')}


False

In [22]:
agent_executor.run(question)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: AdTemplates, Ads, Billboards, BotVectorStores, Brands, BulkTemplateTasks, ButtonRepliedActions, ButtonReplyActions, Categories, Chats, Colors, Conversations, Coupons, Customers, DocumentMessages, ImageHeaders, ImageMessages, Images, InteractiveButtons, InteractiveLists, InteractiveMessages, InteractiveTemplates, ListRepliedActions, ListReplyActions, ListRowButtons, ListSections, LocationMessages, Loyalties, MarketingResponses, Merchants, Messages, MpesaSettings, OrderItems, Orders, ProductCombinations, ProductImages, ProductVariants, ProductVariationOptions, ProductVariations, Products, PromotionCustomers, PromotionProducts, Promotions, Purchases, Redemptions, Rewards, Routes, SaleDetails, Sales, ScheduleTasks, SequelizeMeta, Settings, Sizes, Stores, StripeSettings, TemplateRepliedActions, TemplateReplyActions, TextHeaders, TextMessages, VideoMessages, author_bg, category_bg, comment_bg, migration, pi

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 10.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


  I should try running the query again.
Action: sql_db_query
Action Input: SELECT * FROM Orders WHERE "isPaid" = true AND "createdAt" >= DATE('now', '-1 year');Error: (psycopg2.errors.UndefinedTable) relation "orders" does not exist
LINE 1: SELECT * FROM Orders WHERE "isPaid" = true AND "createdAt" >...
                      ^

[SQL: SELECT * FROM Orders WHERE "isPaid" = true AND "createdAt" >= DATE('now', '-1 year');]
(Background on this error at: https://sqlalche.me/e/20/f405) I should double check my query again.
Action: sql_db_query_checker
Action Input: SELECT * FROM Orders WHERE "isPaid" = true AND "createdAt" >= DATE('now', '-1 year');```sql
SELECT * 
FROM Orders 
WHERE "isPaid" = true 
  AND "createdAt" >= DATE('now', '-1 year');
```The table name in my query is wrong. I should use Orders instead of Order.
Action: sql_db_query
Action Input: SELECT * FROM Orders WHERE "isPaid" = true AND "createdAt" >= DATE('now', '-1 year');Error: (psycopg2.errors.UndefinedTable) relation "orde

'Agent stopped due to iteration limit or time limit.'